In [4]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import plotly.express as px
import plotly.graph_objects as go

In [5]:
tickers = ["AAPL", "MSFT", "TWTR", "IBM"]
stocks = pdr.get_data_yahoo(tickers, "2020-01-01")
stocks.head()

Attributes  Adj Close                                         Close  \
Symbols          AAPL        MSFT       TWTR         IBM       AAPL   
Date                                                                  
2020-01-02  74.096451  157.903473  32.299999  124.142921  75.087502   
2020-01-03  73.376083  155.937286  31.520000  123.152855  74.357498   
2020-01-06  73.960770  156.340393  31.639999  122.932861  74.949997   
2020-01-07  73.612930  154.914886  32.540001  123.015358  74.597504   
2020-01-08  74.797081  157.382446  33.049999  124.042084  75.797501   

Attributes                                          High              ...  \
Symbols           MSFT       TWTR         IBM       AAPL        MSFT  ...   
Date                                                                  ...   
2020-01-02  160.619995  32.299999  135.419998  75.150002  160.729996  ...   
2020-01-03  158.619995  31.520000  134.339996  75.144997  159.949997  ...   
2020-01-06  159.029999  31.639999  134.100006  74.989998  159.100006  ...   
2020-01-07  157.580002  32.540001  134.190002  75.224998  159.669998  ...   
2020-01-08  160.089996  33.049999  135.309998  76.110001  160.800003  ...   

Attributes        Low                   Open                         \
Symbols          TWTR         IBM       AAPL        MSFT       TWTR   
Date                                                                  
2020-01-02  31.959999  134.770004  74.059998  158.779999  32.310001   
2020-01-03  31.260000  133.559998  74.287498  158.320007  31.709999   
2020-01-06  31.160000  133.199997  73.447502  157.080002  31.230000   
2020-01-07  31.719999  133.399994  74.959999  159.320007  31.799999   
2020-01-08  32.349998  133.919998  74.290001  158.929993  32.349998   

Attributes                   Volume                                     
Symbols            IBM         AAPL        MSFT        TWTR        IBM  
Date                                                                    
2020-01-02  135.000000  135480400.0  22622100.0  10721100.0  3148600.0  
2020-01-03  133.570007  146322800.0  21116200.0  14429500.0  2373700.0  
2020-01-06  133.419998  118387200.0  20813700.0  12582500.0  2425500.0  
2020-01-07  133.690002  108872000.0  21634100.0  13712900.0  3090800.0  
2020-01-08  134.509995  132079200.0  27746500.0  14632400.0  4346000.0  

[5 rows x 24 columns]

In [6]:
stocks_price = stocks["Adj Close"]
stocks_price.head()

Symbols,AAPL,MSFT,TWTR,IBM
Date,,,,
2020-01-02,74.096451,157.903473,32.299999,124.142921
2020-01-03,73.376083,155.937286,31.520000,123.152855
2020-01-06,73.960770,156.340393,31.639999,122.932861
2020-01-07,73.612930,154.914886,32.540001,123.015358
2020-01-08,74.797081,157.382446,33.049999,124.042084


In [7]:
log_returns = np.log(stocks_price / stocks_price.shift())
log_returns.head()

Symbols,AAPL,MSFT,TWTR,IBM
Date,,,,
2020-01-02,NaN,NaN,NaN,NaN
2020-01-03,-0.009770,-0.012530,-0.024445,-0.008007
2020-01-06,0.007937,0.002582,0.003800,-0.001788
2020-01-07,-0.004714,-0.009160,0.028048,0.000671
2020-01-08,0.015958,0.015803,0.015551,0.008312


In [8]:
weight = np.random.random(4)
weight /= weight.sum()
weight

array([0.22617573, 0.22505589, 0.21416697, 0.33460142])

In [9]:
expected_returns = np.sum(log_returns.mean() * weight) * 252
expected_returns

0.2388844993234265

In [10]:
expected_volatility =  np.dot(weight.T, np.dot(log_returns.cov() * 252, weight)) ** 0.5
expected_volatility

0.31971617785575773

In [11]:
sharp_ratio = expected_returns / expected_volatility
sharp_ratio

0.7471767644839072

In [12]:
# Monte Carlo 
n = 10_000
weights = np.zeros((n, 4))
expected_returns = np.zeros(n)
expected_volatility = np.zeros(n)
sharp_ratio = np.zeros(n)

for i in range(n):
    weight = np.random.random(4)
    weight /= weight.sum()
    
    weights[i] = weight
    expected_returns[i] = np.sum(log_returns.mean() * weight) * 252
    expected_volatility[i] =  np.sqrt(np.dot(weight.T, np.dot(log_returns.cov() * 252, weight)))
    sharp_ratio[i] = expected_returns[i] / expected_volatility[i]


In [ ]:
sharp_ratio.max(), sharp_ratio.argmax()

(1.1439893104795125, 2249)

In [ ]:
weights[sharp_ratio.argmax()]

array([0.28806698, 0.6995455 , 0.00817749, 0.00421004])

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=expected_volatility,
                         y=expected_returns,
                         mode="markers",
                         marker=dict(color=sharp_ratio,
                                     size=5)))
fig.add_trace(go.Scatter(x=[expected_volatility[sharp_ratio.argmax()]],
                         y=[expected_returns[sharp_ratio.argmax()]],
                         name="",
                         showlegend=False,
                         mode="markers",
                         hovertemplate=f"Best Sharpe Ratio = {sharp_ratio.max():.3f}<br>" +
                                        "Expected Volatility = %{x}<br>" +
                                        "Expected Returns = %{y}", 
                         marker=dict(size=5,
                                     symbol="x-dot",
                                     line=dict(width=1,
                                               color="red"))))

fig.show()

In [ ]:
|